In [ ]:
import torch as th
import einops as eo

In [ ]:
x = th.ones((3, 1, 5))
y = th.ones((1, 4, 5))

z = x + y
print(z.shape)

In [ ]:
x = th.ones((8, 2, 6))
y = th.ones((1, 2, 1))
z = x + y
print(z.shape)

In [ ]:
tens = th.arange(2, 8) 
print(tens.shape)

tens = eo.rearrange(tens, "(w w1) -> w w1", w1=2)
print(tens.shape)
print(tens)

In [ ]:
temps = th.tensor([71, 72, 70, 75, 71, 72, 70, 75, 80, 85, 80, 78, 72, 83]).float()
expected = [-0.571, 0.429, -1.571, 3.429, -0.571, 0.429, -1.571, -4.0, 1.0, 6.0, 1.0, -1.0, -7.0, 4.0]

weekly = eo.reduce(temps, "(week days) -> week", days=7, reduction="mean")

tp2 = temps.view(2, 7)
we2 = weekly
print(tp2)
print(we2)
print(tp2.shape)
print(we2.shape)

ans = tp2 - we2
print(ans.shape)
ans = ans.reshape(-1)
print(ans.shape)
print(ans)


In [ ]:
avg = eo.reduce(temps, "(week days) -> week", days=7, reduction="mean")
# reduce over days using t.std: we obtain (week=2) -> (std_w1, std_w2)
std = eo.reduce(temps, "(week days) -> week", days=7, reduction=th.std)
res = (temps.view(2, 7) - avg.view(2, 1)) / std.view(2, 1)

print(res.shape)

In [ ]:
probs = [0.2, 0.3, 0.5]
probs_cumsum = th.cumsum(th.tensor(probs), dim=0)
print(probs_cumsum)

samples = th.rand(10)
print(samples)

print((samples.unsqueeze(-1) > probs_cumsum).sum(dim=-1))


In [ ]:
test = th.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
print(test.shape)
print(test[0, 1])       # row, col

# gather example: get the [2, 3], [4, 5], [7, 9]
indices = th.tensor([[1, 2], [0, 1], [0, 2]])       # shape (3, 2)

# For each "row" along dim=1, get the elements at those specified indices
# if an element is another "array" then we just get
gathered_0 = th.gather(test, dim=0, index=indices)
print(gathered_0)
# dim = 0; out[i, j] = input[indices[i, j], j]
# ex: out[0, 0] = input[indices[0, 0,], 0] = input[1, 0] = 4

gathered_1 = th.gather(test, dim=1, index=indices)
print(gathered_1)
# dim = 1; out[i, j] = input[i, indices[i, j]]
# ex out[0, 0] = input[0, indices[0, 0]] = input[0, 1] = 2

src = th.rand(5)
index = th.tensor([0, 0, 0, 1, 1, 1, 2, 2, 2])
print(th.gather(src, 0, index))

In [ ]:
# 2 dim. tensor with magnitude 3 in first dim, magnitude 5 in second dim
mat_2d = th.arange(15).view(3, 5)
coords_2d = th.tensor([[0, 1], [0, 4], [1, 4]])

th.gather(mat_2d, 1, coords_2d)


In [ ]:
mat = th.tensor([[-1000, -1000, -1000, -1000], [1000, 1000, 1000, 1000]])
print(mat)
print(mat.shape)

max = mat.max(dim=-1, keepdim=True).values
print(max)
print(max.shape)

out = mat - max 
print(out)
print(out.shape)

In [ ]:
matrix = th.rand((10, 20))
print(matrix.shape)

C = matrix.max(dim=-1).values
print(C.shape)

C = eo.rearrange(C, "b -> b 1")
print(C.shape)


res = out - C 
print(res.shape)

denum = res.sum(dim=-1, keepdim=True)
print(denum.shape)

print(res / denum)

In [ ]:
import torch as t 
from torch import Tensor
import einops


def batched_logsoftmax(matrix: Tensor) -> Tensor:
    """Compute log(softmax(row)) for each row of the matrix.

    matrix: shape (batch, n)

    Return: (batch, n).

    Do this without using PyTorch's logsoftmax function.
    For each row, subtract the maximum first to avoid overflow if the row contains large values.
    """
    C = matrix.max(dim=-1).values                             # shape (batch, )
    # note: einops.rearrange(C, "b -> b 1")) goes from (batch, ) -> (batch, 1) so we can broadcast
    num = t.exp(matrix - einops.rearrange(C, "b -> b 1"))     # shape (batch, n)
    denum = num.sum(dim=-1, keepdim=True)                     # shape (batch, 1) (so we can broadcast)
    return t.log(num / denum)                                 # (batch, n) / (batch, 1) 



logits = t.tensor([[float("-inf"), float("-inf"), 0], [1 / 3, 1 / 3, 1 / 3], [float("-inf"), 0, 0]])
# true labels per batch
true_labels = t.tensor([2, 0, 0])

# Turn logits into probabilities
probs = batched_logsoftmax(logits)
print(probs.shape)

# get the log probability of the true labels for each batch
print(true_labels.shape)
print(true_labels.unsqueeze(-1).shape)
print(logits)
print(true_labels.unsqueeze(-1))
log_prob_true = t.gather(probs, dim=1, index=true_labels.unsqueeze(-1)).squeeze(-1)
print(log_prob_true)

torch.Size([3, 3])
torch.Size([3])
torch.Size([3, 1])
tensor([[  -inf,   -inf, 0.0000],
        [0.3333, 0.3333, 0.3333],
        [  -inf, 0.0000, 0.0000]])
tensor([[2],
        [0],
        [0]])
tensor([ 0.0000, -1.0986,    -inf])
tensor(nan)


In [70]:
matrix = t.arange(15).view((5, 3))
row_indexes = t.tensor([0, 2, 1, 0])

print(matrix.shape)
print(row_indexes.shape)

# input (5, 3), index (4,), output (4, 3)
selected_rows = t.gather(matrix, dim=0, index=row_indexes.unsqueeze(-1))
print(selected_rows)

torch.Size([5, 3])
torch.Size([4])
tensor([[0],
        [6],
        [3],
        [0]])
